# Model Utils

This is a notebook for all common model functions.

In [1]:
import tensorflow as tf
import numpy as np
import math

## Utils

In [2]:
def get_variable(name, shape, alpha=0.01):
    initial = alpha * tf.random_normal(shape)
    return tf.Variable(initial, name=name)

In [3]:
def get_optimizer(labels, logits, learning_rate = 10**-7):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam').minimize(loss)
    return loss, optimizer

In [4]:
def get_accuracy(labels, logits):
    prediction = tf.cast(tf.greater(logits, 0.5), dtype=tf.float32)
    correct = tf.equal(labels, prediction)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    return accuracy

In [5]:
def random_mini_batches(X1, X2, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X1 -- input data part 1, of shape (number of examples, ...)
    X2 -- input data part 1, of shape (number of examples, ...)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (number of examples, 1)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X1.shape[0]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X1 = X1[permutation]
    shuffled_X2 = X2[permutation]
    shuffled_Y = Y[permutation].reshape(m, 1)

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X1 = shuffled_X1[k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch_X2 = shuffled_X2[k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch_Y = shuffled_Y[k*mini_batch_size : (k+1)*mini_batch_size]
        mini_batch = (mini_batch_X1, mini_batch_X2, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X1 = shuffled_X1[num_complete_minibatches*mini_batch_size : ]
        mini_batch_X2 = shuffled_X2[num_complete_minibatches*mini_batch_size : ]
        mini_batch_Y = shuffled_Y[num_complete_minibatches*mini_batch_size : ]
        mini_batch = (mini_batch_X1, mini_batch_X2, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches